In [6]:
import os, glob
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [7]:
import nibabel as nib
from skimage.measure import label, regionprops

def calculate_iou(seg1, seg2):
    # load Nifti files
    nii1 = nib.load(seg1)
    nii2 = nib.load(seg2)
    data1 = nii1.get_fdata()
    data2 = nii2.get_fdata()

    # calculate intersection and union volumes
    intersection = (data1 > 0) & (data2 > 0)
    union = (data1 > 0) | (data2 > 0)
    iou = intersection.sum() / union.sum()
    return iou

def calculate_dice(seg1, seg2):
    # load Nifti files
    nii1 = nib.load(seg1)
    nii2 = nib.load(seg2)
    data1 = nii1.get_fdata()
    data2 = nii2.get_fdata()

    # calculate dice coefficient
    intersection = (data1 > 0) & (data2 > 0)
    dice = 2.0 * intersection.sum() / (data1.sum() + data2.sum())
    return dice

In [9]:
"""import matplotlib.pyplot as plt
# 경로 설정
seg1_path = 'C:/RIB/predictions/RibFrac542-image_pred.nii'
seg2_path = 'C:/RIB/Ribseg/test/RibFrac542-rib-seg.nii'

# IoU 계산
iou = calculate_iou(seg1_path, seg2_path)

# Dice 계수 계산
dice = calculate_dice(seg1_path, seg2_path)

# Load Nifti files and calculate IoU and Dice
iou = calculate_iou(seg1_path, seg2_path)
dice = calculate_dice(seg1_path, seg2_path)

# Create bar chart
labels = ['IoU', 'Dice']
values = [iou, dice]
plt.bar(labels, values)
plt.ylabel('Similarity')
plt.ylim(0, 1)
plt.show()
print('Dice: {:.4f}'.format(dice))
print('IoU: {:.4f}'.format(iou))"""

"import matplotlib.pyplot as plt\n# 경로 설정\nseg1_path = 'C:/RIB/predictions/RibFrac542-image_pred.nii'\nseg2_path = 'C:/RIB/Ribseg/test/RibFrac542-rib-seg.nii'\n\n# IoU 계산\niou = calculate_iou(seg1_path, seg2_path)\n\n# Dice 계수 계산\ndice = calculate_dice(seg1_path, seg2_path)\n\n# Load Nifti files and calculate IoU and Dice\niou = calculate_iou(seg1_path, seg2_path)\ndice = calculate_dice(seg1_path, seg2_path)\n\n# Create bar chart\nlabels = ['IoU', 'Dice']\nvalues = [iou, dice]\nplt.bar(labels, values)\nplt.ylabel('Similarity')\nplt.ylim(0, 1)\nplt.show()\nprint('Dice: {:.4f}'.format(dice))\nprint('IoU: {:.4f}'.format(iou))"

In [12]:
# 결과를 저장할 딕셔너리 생성
iou_dice_results = {}

# 541번부터 669번까지의 이미지에 대한 IoU와 Dice 계수 계산
for image_number in range(541, 670):
    # segmentation과 label 데이터 경로 설정
    seg_path = f'C:/RIB/predictions/RibFrac{image_number}-image_pred.nii'
    label_path = f'C:/RIB/Ribseg/test/RibFrac{image_number}-rib-seg.nii'
    
    try:
        # IoU와 Dice 계수 계산
        iou = calculate_iou(seg_path, label_path)
        dice = calculate_dice(seg_path, label_path)

        # 결과를 딕셔너리에 저장
        iou_dice_results[f'이미지 {image_number}'] = {'IoU': iou, 'Dice': dice}
        print('Dice: {:.4f}'.format(dice))
        print('IoU: {:.4f}'.format(iou))
    except FileNotFoundError:
        print(f'{seg_path} 파일을 찾을 수 없습니다. 해당 이미지를 스킵합니다.')

# IoU와 Dice 값을 리스트로 추출
iou_list = [iou_dice_results[f'이미지 {image_number}']['IoU'] for image_number in range(541, 670) if f'이미지 {image_number}' in iou_dice_results]
dice_list = [iou_dice_results[f'이미지 {image_number}']['Dice'] for image_number in range(541, 670) if f'이미지 {image_number}' in iou_dice_results]

# 막대 그래프로 시각화
"""
labels = [f'이미지 {image_number}' for image_number in range(541, 670) if f'이미지 {image_number}' in iou_dice_results]
x = range(len(labels))

plt.bar(x, iou_list, label='IoU')
plt.bar(x, dice_list, label='Dice')
plt.xticks(x, labels, rotation=90)
plt.ylabel('Similarity')
plt.ylim(0, 1)
plt.legend()
plt.show()
"""
# IoU와 Dice 값을 리스트로 추출
iou_list = [iou_dice_results[f'이미지 {image_number}']['IoU'] for image_number in range(541, 670) if f'이미지 {image_number}' in iou_dice_results]
dice_list = [iou_dice_results[f'이미지 {image_number}']['Dice'] for image_number in range(541, 670) if f'이미지 {image_number}' in iou_dice_results]  

# IoU와 Dice 평균값 계산
iou_mean = sum(iou_list) / len(iou_list)
dice_mean = sum(dice_list) / len(dice_list)

# 평균값 출력
print('평균 Dice: {:.4f}'.format(dice_mean))
print('평균 IoU: {:.4f}'.format(iou_mean))

Dice: 0.8318
IoU: 0.7121
Dice: 0.8295
IoU: 0.7086
Dice: 0.8427
IoU: 0.7282
Dice: 0.7882
IoU: 0.6504
Dice: 0.8446
IoU: 0.7310
Dice: 0.8772
IoU: 0.7812
Dice: 0.8841
IoU: 0.7923
Dice: 0.7718
IoU: 0.6284
Dice: 0.8524
IoU: 0.7428
C:/RIB/predictions/RibFrac550-image_pred.nii 파일을 찾을 수 없습니다. 해당 이미지를 스킵합니다.
Dice: 0.8195
IoU: 0.6942
Dice: 0.8847
IoU: 0.7933
Dice: 0.8368
IoU: 0.7194
Dice: 0.7746
IoU: 0.6322
Dice: 0.8425
IoU: 0.7279
Dice: 0.9018
IoU: 0.8211
Dice: 0.8178
IoU: 0.6917
C:/RIB/predictions/RibFrac558-image_pred.nii 파일을 찾을 수 없습니다. 해당 이미지를 스킵합니다.
Dice: 0.8602
IoU: 0.7547
Dice: 0.7922
IoU: 0.6559
Dice: 0.8028
IoU: 0.6705
Dice: 0.8619
IoU: 0.7573
Dice: 0.7725
IoU: 0.6293
Dice: 0.7951
IoU: 0.6599
Dice: 0.8227
IoU: 0.6987
Dice: 0.7685
IoU: 0.6241
Dice: 0.7806
IoU: 0.6402
Dice: 0.8325
IoU: 0.7130
C:/RIB/predictions/RibFrac569-image_pred.nii 파일을 찾을 수 없습니다. 해당 이미지를 스킵합니다.
C:/RIB/predictions/RibFrac570-image_pred.nii 파일을 찾을 수 없습니다. 해당 이미지를 스킵합니다.
C:/RIB/predictions/RibFrac571-image_pred.nii 파일을 찾